In [ ]:
import numpy as np
np.seterr(all='ignore')

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

from my_functions import *
from load_mocks import ensemble_mock

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67
filter_tags = load_filter_tags()

In [ ]:
name_qso = 'QSO_double_train_minijpas_DR16_0'
name_gal = 'GAL_LC_minijpas_0'
name_sf = 'LAE_12.5deg_z2-4.25_train_minijpas_0'

pm_flx, pm_err, zspec, EW_lya, L_lya, is_qso, is_sf, is_gal, _, _, R_ang =\
    ensemble_mock(name_qso, name_gal, name_sf)

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

N_sources = pm_flx.shape[1]
N_sources

In [ ]:
count_true(is_gal & (zspec < 4) & (mag < 24))

In [ ]:
PD_z_Arr = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5])
PD_counts_Arr = np.array([975471, 2247522, 1282573, 280401, 31368, 4322])

fig, ax = plt.subplots(figsize=(7, 6))

z_min = 0
z_max = 4.5
n_bins = 40
area_out = 400

bins = np.linspace(z_min, z_max, n_bins)
bins_c = [(bins[i] + bins[i + 1]) / 2 for i in range(len(bins) - 1)]
bin_size = (z_max - z_min) / (n_bins - 1)

z_hist, _ = np.histogram(zspec[is_qso], bins=bins)

ax.step(bins_c, z_hist / bin_size / area_out, label='z out')

area_in = 10_000
ax.plot(PD_z_Arr, PD_counts_Arr / 1 / area_in, marker='s', label='z in')

ax.set_yscale('log')
ax.set_ylabel('$\Delta z^{-1}\,\mathrm{deg}^{-2}$', fontsize=15)
ax.set_xlabel('z', fontsize=15)
ax.set_ylim((1, 1e3))
ax.legend(fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(15, 25, 30)
bins_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
h, _ = np.histogram(mag[is_qso], bins=bins)
bin_size = (25 - 15) / (50 - 1)

ax.step(bins_c, h / bin_size / 400)

ax.set_xlabel('r (magAB)', fontsize=15)
ax.set_ylabel('$\Delta r^{-1}\,\mathrm{deg}^{-2}$', fontsize=15)
ax.set_yscale('log')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

ax.hist(L_lya[is_qso & (mag < 23)], bins=np.linspace(42, 45.5, 50), cumulative=True, density=True)
# ax.set_yscale('log')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

L_min = 42
L_max = 46
n_bins = 30
z_min = 2.2
z_max = 3.5
area = 400

bins = np.linspace(L_min, L_max, n_bins)
bins_c = [(bins[i] + bins[i + 1]) / 2 for i in range(len(bins) - 1)]
bin_size = (L_max - L_min) / (n_bins - 1)
volume = z_volume(z_min, z_max, area)

z_mask = (zspec > z_min) & (zspec < z_max)
L_hist, _ = np.histogram(L_lya[is_qso & z_mask], bins=bins)
L_hist_ewmin, _ = np.histogram(L_lya[is_qso & z_mask & (EW_lya > 30)], bins=bins)

ax.step(bins_c, L_hist / volume / bin_size, label='LF out')

Lx = np.logspace(L_min, L_max, 10000)
# Daniele's LF
# phistar1 = 3.33e-6
# Lstar1 = 44.65
# alpha1 = -1.35
# Zhang's LF
phistar1 = 10 ** -5.85
Lstar1 = 44.6
alpha1 = -1.2
Phi = schechter(Lx, phistar1, 10 ** Lstar1, alpha1) * Lx * np.log(10)

ax.plot(np.log10(Lx), Phi, label='LF in')

ax.set_yscale('log')
ax.set_ylabel(r'$\Phi$ [Mpc$^{-1}$ ($\Delta\log L$)$^{-1}$]', fontsize=15)
ax.set_xlabel(r'$\log L$', fontsize=15)
ax.set_ylim((1e-8, 3e-4))
ax.legend(fontsize=15)

plt.show()

In [ ]:
ew0_cut = 30
ew_other = 100

# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0_cut)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, cont_est_lya, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    ew_other, obs=True)
other_lines = identify_lines(line_other, cont_est_other, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.16

# %xdel cont_est_other
%xdel cont_err_other

In [ ]:
mag_min = 17
mag_max = 24

nb_min = 5
nb_max = 20
# nb_min = 16
# nb_max = 23

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1
print(f'z interval: ({z_min:0.2f}, {z_max:0.2f})')

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
ew_cut = EW_lya > ew0_cut
mag_cut = (mag > mag_min) & (mag < mag_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr
)
nice_lya = (nice_lya & z_cut & mag_cut)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))
ax.hist(R_ang * 3600, np.linspace(1e-8, 2))
ax.set_xlabel('R_ang (arcsec)', fontsize=15)
ax.set_ylabel('N', fontsize=15)
ax.set_yscale('log')
plt.show()

In [ ]:
def do_this():
    z_gal = zspec[nice_lya & is_gal & (mag < 23)]
    z_lya = z_Arr[nice_lya & is_gal & (mag < 23)]

    w_0 = w_lya * (1 + z_lya) / (1 + z_gal)

    fig, ax = plt.subplots(figsize=(7, 6))

    ax.hist(w_0, bins=np.linspace(1000, 4600, 30))

    plt.show()
    
    fig, ax = plt.subplots(figsize=(7, 6))

    ax.hist(z_gal, np.linspace(0, 3, 20))
    
    print(count_true(z_gal))

    plt.show()

do_this()

In [ ]:
EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
)

# ML_predict_mask = (mag < 23) & (L_Arr > 0)
# L_Arr[ML_predict_mask] = ML_predict_L(
#     pm_flx[:, ML_predict_mask], pm_err[:, ML_predict_mask],
#     z_Arr[ML_predict_mask], L_Arr[ML_predict_mask], 'RFmag15-23'
# )

# ML_predict_mask = (mag > 23) & (L_Arr > 0)
# L_Arr[ML_predict_mask] = ML_predict_L(
#     pm_flx[:, ML_predict_mask], pm_err[:, ML_predict_mask],
#     z_Arr[ML_predict_mask], L_Arr[ML_predict_mask], 'RFmag23-23.5'
# )

In [ ]:
selected = np.random.permutation(
    np.where(
        nice_lya & ~nice_z & is_gal & z_cut
    )[0]
)
print(count_true(selected))

# qso_lines = [1025.7220, 1397.61, 1549.48, 1908.73, 2799.12]
# Actually gal lines
qso_lines = [4861, 3727, 5007, 6300, 6548, 6730, 1580, 2050]

for i, src in enumerate(selected):
    if i == 10: break
    print(z_cut[src])
    print(f'L_lya = {L_lya[src]}')
    print(f'L_Arr = {L_Arr[src]}')
    print(f'EW0 = {EW_lya[src]}')
    print(f'is_qso = {is_qso[src]}')
    print(nice_lya[src], nice_z[src])

    lya_obs_w = w_central[lya_lines[src]]
    other_lines_w = [w_central[i] for i in other_lines[src]]
    lya_real_w = w_lya * (1 + zspec[src])

    fig = plt.figure(figsize=(10, 5))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    ax.axvline(lya_obs_w, linestyle='--', color='r', label='Retrieved Lya line')
    ax.axvline(lya_real_w, color='g', label='Real Lya line')
    for w in other_lines_w:
        ax.axvline(w, linestyle='--', color='dimgray')
    for w in qso_lines:
        qso_obs_w = w * (1 + zspec[src])
        ax.axvline(qso_obs_w, linestyle=':', color='orange')
    print(other_lines[src])

    ax.plot(w_central[:56], cont_est_lya[:, src])
    ax.plot(w_central[:56], cont_est_other[:, src], ls='--')

    ax.set_title(f'zspec={zspec[src]:0.3f}, z_Arr={z_Arr[src]:0.3f}')
    ax.legend()

    # ax.set_ylim((-1e-17, 1e-17))
    ax.set_xlim((3000, 10000))

    plt.show()